### Práctica de Web Scraping

Se va a considerar una de las páginas vista en la teoría que contiene fotos de trenes:https://www.vialibre-ffe.com/multi_foto.asp?cs=mult. En esta página aparece un menú con varios grupos de fotos de trenes. A su vez, cada grupo contiene un conjunto de subgrupos de fotos. Por ejemplo, el grupo __Renfe operadora__ se encuentra en la página https://www.vialibre-ffe.com/multi_ind_fotos.asp?cat=mu01.

In [ ]:
from IPython.display import Image
Image(filename='Captura1.png')

Dentro de este grupo por ejemplo se encuentra el subgrupo __Serie120 en Irún-Hendaya__ en la página https://www.vialibre-ffe.com/multi_galeria.asp?gal=524. 

In [ ]:
from IPython.display import Image
Image(filename='Captura2.png')

En dicho subgrupo se encuentran las imágenes de dicho subgrupo. Por ejemplo, a continuación, se muestra una de las imágenes del subgrupo __Serie120 en Irún-Hendaya__

In [ ]:
from IPython.display import Image
Image(filename='Captura1.png')
#for i in range(1,4):
#    Image(filename='Captura'+str(i)+'.png')

Las imágenes contenidas en dicha página se pueden encontrar en las etiquetas __&lt;li> &lt;img ...__ que son hijas de la etiqueta __&lt;ul class="pgwSlideshow">__. Por ejemplo en la página de ejemplo, la primera imagen de dicha página es:

   __&lt;li>&lt;img src="multimedia/galerias/IRUN120/2Alvia_120.jpg" alt="" data-description="">&lt;/li>__
   
A continuación, se puede ver el contenido html de esa página.

In [ ]:
import requests
url="https://www.vialibre-ffe.com/multi_galeria.asp?gal=524"
r = requests.get(url)
html = r.text
print(html)

Se pide:

 1. Crear un programa en __Python__ que muestre un menú con los grupos de fotos, y que pregunte al usuario que grupo de fotos quiere visitar. Se puede asociar a cada grupo un número, y solicitar al usuario que introduzca por teclado el número del grupo. A continuación, se mostrará los subgrupos de ese grupo, y se le preguntará nuevamente al usuario que subgrupo de fotos quiere procesar. Igual que antes, se puede asociar a cada subgrupo un número, y que introduzca por teclado el número del subgrupo. Como resultado se creará un directorio en el disco local para el subgrupo elegido y en el directorio se bajarán las imágenes de los trenes. Se imprimirá por pantalla las urls de las imágenes que se están bajando. Toda la información utilizada en el programa, debe ser extraida de las páginas consideradas usando BeautifulSoup[6 puntos]
 
 2. Crear un minibuscador en __Python__ que pregunte al usuario un conjunto de palabras clave, y  recorra el sitio web buscando todos los subgrupos en cuyo título aparezca alguna de las palabras claves. Como resultado debe mostrar los subgrupos encontrados, listando el nombre y la url de cada subgrupo [4 puntos]

# Normas de entrega

* Fecha tope de entrega: 18/11/2017
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre WebScraping_GrupoX donde X será el número de grupo correspondiente.


In [2]:
import os
import urllib
import urllib.request
import re
from bs4 import BeautifulSoup


# **** EJERCICIO 1 ****
def explorarFerrocarriles():
    print('**** EJERCICIO 1 ****')
    print()
    #URL base de la pagina web
    url="https://www.vialibre-ffe.com"
    #HTML de la pagina en la que empezamos a trabajar
    html=urllib.request.urlopen("https://www.vialibre-ffe.com/multi_foto.asp?cs=mult")
    soup=BeautifulSoup(html.read(),'html.parser')
    primero=soup.find_all("a") # Buscamos todos los links
    # Regex para reconocer las URLs que nos interesan
    patron=re.compile(r"\/multi\_ind\_fotos\.asp\?cat=mu.*")
    # Lista que contendra los nombres de los links
    listaNombres=[]
    # Lista que contendra las URLs
    listaURL=[]
    # Parsea el HTML de la primera pagina para buscar las URLs que coinciden
    # con el patron; despues los guarda en listas
    for etiqueta in primero:
        nomb = etiqueta.contents
        link = etiqueta.get('href',None)
        matches=patron.search(link)
        # Si el patron coincide
        if matches != None:
            listaNombres.append(nomb[0])
            listaURL.append(link)
    # Muestra por pantalla las opciones disponibles
    print("------------------------------------Menú Principal:------------------------------------")
    con = 0
    for nomb in listaNombres:
        con = con + 1
        print(str(con) + ". " + nomb)
    # Pide la eleccion del usuario por pantalla y comprueba que la opcion es valida
    opt = input("Elige una opcion: ")
    try:
        opt = int(opt)
    except:
        opt = 0
        print("Por favor, introduce solo números")
    while (opt<1 or opt >len(listaNombres)):
        print("No existe esa opción. Elige otra.")
        print()
        opt = input("Elige una opcion: ")
        try:
            opt = int(opt)
        except:
            opt = 0
            print("Por favor, introduce solo números")
    #Creamos una carpeta con el nombre de la opcion introducida y me meto en ella
    #si ya existe esa carpeta no la creo solo me meto en ella
    if os.path.exists(str(listaNombres[int(opt)-1])) is False:
        os.makedirs(str(listaNombres[int(opt)-1]))
    os.chdir(str(listaNombres[int(opt)-1]))
    # Elige el link sobre el que trabajar de la lista anterior
    link = url+listaURL[int(opt)-1]
    # Se repite de nuevo el proceso para el nuevo link, que es
    # un submenu de opciones.
    html=urllib.request.urlopen(link)
    soup=BeautifulSoup(html.read(),'html.parser')
    segundo=soup.find_all("a")
    # El patron para el submenu es diferente; asi como las listas.
    patron2=re.compile(r"multi\_galeria\.asp\?gal=.*")
    listaURL2 = []
    listaNombres2 = []
    # Añade a la lista los links que coinciden con el patron buscado y
    # no esten ya en la lista (algunos links aparecian duplicados)
    for etiqueta in segundo:
        link = etiqueta.get('href',None)
        matches=patron2.search(link)
        if matches != None and link not in listaURL2:
            listaURL2.append(link)
            listaNombres2.append(etiqueta.contents[1].get('alt',None))
    # Muestra las nuevas opciones por pantalla
    print("------------------------------------Menú para "+str(listaNombres[int(opt)-1])+":------------------------------------")
    con = 0
    for nomb in listaNombres2:
        con = con + 1
        print(str(con) + ". " + nomb)
    # Pide la eleccion del usuario por pantalla y comprueba que la opcion es valida
    opt = input("Elige una opcion: ")
    try:
        opt = int(opt)
    except:
        opt = 0
        print("Por favor, introduce solo números")
    while (opt<1 or opt >len(listaNombres2)):
        print("No existe esa opción. Elige otra.")
        print()
        opt = input("Elige una opcion: ")
        try:
            opt = int(opt)
        except:
            opt = 0
            print("Por favor, introduce solo números")
    #Creo otra carpeta para indicar la otra opcion, en esata será donde guarde las fotos
    #Si ya existe esa carpeta solo me meto en ella, pero sin crearla
    if os.path.exists(str(listaNombres2[int(opt)-1])) is False:
        os.makedirs(str(listaNombres2[int(opt)-1]))
    os.chdir(str(listaNombres2[int(opt)-1]))
    
    #Tengo el link donde estan las fotos que quiero descargar
    link = url+'/'+listaURL2[int(opt)-1]
    #Cogo el html de esa pagina, que es donde estan las imagenes que quiero
    html=urllib.request.urlopen(link).read()
    soup=BeautifulSoup(html, 'html.parser')
    #Guardo en una lista las etiquetas ul de la pagina
    ul=[]
    ul=soup.find_all("ul")
    #Consigo la ul <ul class="pgwSlideshow">,que es la ultima que aparece en el html
    li =ul[len(ul)-1]
    #Guardo en una lista las etiquetas li que aparecen (que cada una de ellas es una imagen)
    lo=li.find_all("li")
    j=1 #contador para ir nombrando a las imagenes una vez las guardemos
    print("------------------------------------URLs de las imágenes:------------------------------------")
    #Recorro cada imagen
    for imagen in lo:
        #Consigo en que posicion esta el link de la imagen
        posInicio=str(imagen).find('multimedia')
        posFinal=str(imagen).find('"/>')
        imagenUrl=""
        #Recorro eso para obtener la direccion de la foto
        while (posInicio!=posFinal):
            imagenUrl=imagenUrl+str(imagen)[posInicio]
            posInicio=posInicio+1
        #Url de cada imagen
        urlEntera=url+'/'+imagenUrl
        print(urlEntera)#url entera a mostrar 
        #Descargo la foto
        try:
            img = urllib.request.urlopen(urlEntera)
            manf=open("imagen"+str(j)+".jpg","wb")
            while True:
                info=img.read(100000)
                if len(info)<1:
                    break
                manf.write(info)
            manf.close()
            j=j+1
        except:
            error=0
    print("------------------------------------Imágenes descargadas------------------------------------")
    #Vuelvo a la carpeta raiz donde estaba
    os.chdir("../../")
    print()
    print()
    
    
    
# **** EJERCICIO 2 ****
def miniBuscador():
    print('**** EJERCICIO 2 ****')
    print()
    #URL base de la pagina web
    urlBase = "https://www.vialibre-ffe.com"
    #HTML de la pagina en la que empezamos a trabajar
    html = urllib.request.urlopen("https://www.vialibre-ffe.com/multi_foto.asp?cs=mult")
    soup = BeautifulSoup(html.read(),'html.parser')
    #Obtenemos las etiquetas donde están los enlaces a los grupos
    etiquetasGrupos = soup.find_all(href=re.compile("\/multi\_ind\_fotos\.asp\?cat=mu.*"))
  
    #Pedimos al usuario las palabras que quiere buscar
    entrada = []
    while(len(entrada) == 0):
        entrada = input("Introduzca las palabras clave a buscar (separadas por espacios): ")
        #Las separamos
        entrada = entrada.strip()
        palabrasClave = entrada.split()
        if (len(entrada) == 0):
            print('*Por favor, introduzca al menos una palabra para poder realizar la búsqueda*')
        print()
    
    numResultados = 0;
    #Por cada una de las palabras introducidas
    for palabra in palabrasClave:
        #Buscamos en cada uno de los grupos de la web
        for grupo in etiquetasGrupos:
            html = urllib.request.urlopen(urlBase+grupo.get("href",None))
            soup = BeautifulSoup(html.read(),'html.parser')
            #Cogemos todas las etiquetas <p>
            subgrupos = soup.find_all('p')
            for parrafos in subgrupos:
                #Y dentro de ellas nos quedamos con las que tienen como hija una
                #etiqueta <a>, ya que solamente coincide con esta característa los
                #subgrupos que buscamos
                subgrupo = parrafos.find('a')
                #Si el título del subfrupo contiene la palabra buscada
                if (subgrupo != None and palabra.lower() in subgrupo.string.lower()):
                    #Mostramos el nombre y la url
                    print(subgrupo.string, ':', urlBase+'/'+subgrupo.get("href",None))
                    numResultados += 1       
    print()
    print('Se han encontrado', numResultados, 'resultado(s)')
    
    
    
# ****LLAMADAS A LAS FUNCIONES****

#Ejercicio 1
explorarFerrocarriles()  
#Ejercicio 2
miniBuscador()

**** EJERCICIO 1 ****

------------------------------------Menú Principal:------------------------------------
1. Renfe Operadora
2. Ferrocarriles autonómicos
3. Metropolitanos
4. Tranvías
5. Otros operadores
6. Infraestructuras
7. Instalaciones
8. Estaciones
9. Internacional
10. Históricas
11. Museos y trenes
12. Varios
Elige una opcion: 3
------------------------------------Menú para Metropolitanos:------------------------------------
1. Metro de Granada
2. Metro Málaga
3. Serie 300 de Metro de Barcelona
4. Unidades del Metro de Málaga
5. Coche serie 100 de TMB
6. Los trenes de cercanías de CAF para Esmirna (Turquía)
7. Metro Valencia
8. Diseño coches línea 9 del Metro de Barcelona
9. Metro Automatico de Laussane (Suiza)
10. Metro de Pekin
11. Metro de Sevilla
12. Libro "Metro" de Alberto Schommer
13. Metro Barcelona L9
14. Metro de El Cairo
15. Los nuevos trenes adaptados de Metro de Madrid
16. Nueva estación de Guinardó/HospitalSant Pau
17. Metro de Palma de Mallorca
18. El Metro d